<a href="https://colab.research.google.com/github/liuyao12/imagenette_experiments/blob/master/Woof_ResNet_separable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ResNet with separable convolution

> depth multiplier + Ranger + Mish + MaxBlurPool + restrick

# setup and imports

In [ ]:
# pip install kornia

In [1]:
pip install git+https://github.com/ayasyrev/model_constructor

  Cloning https://github.com/ayasyrev/model_constructor to /tmp/pip-req-build-iynaaz0_
  Running command git clone -q https://github.com/ayasyrev/model_constructor /tmp/pip-req-build-iynaaz0_
  Created wheel for model-constructor: filename=model_constructor-0.1.1-cp36-none-any.whl size=23583 sha256=9bd75397145844dc20462e63cae1e76759245e88a6aede60e503dd40bdd5323c
  Stored in directory: /tmp/pip-ephem-wheel-cache-qdcgq87o/wheels/5b/92/65/8093a307d5802f41f4a8776b40bc12b558e75a2a906ae8b683
Successfully built model-constructor


In [2]:
pip install git+https://github.com/ayasyrev/imagenette_experiments

  Cloning https://github.com/ayasyrev/imagenette_experiments to /tmp/pip-req-build-6m_k9i6o
  Running command git clone -q https://github.com/ayasyrev/imagenette_experiments /tmp/pip-req-build-6m_k9i6o
  Created wheel for imagenette-experiments: filename=imagenette_experiments-0.0.1-cp36-none-any.whl size=15332 sha256=809b1bfde2a920a8dc9abbec08abc37aa5d1625f4343be5a615fabd7a8aa35e8
  Stored in directory: /tmp/pip-ephem-wheel-cache-fu24r8kd/wheels/af/99/98/2212941f45a18bf6d191f56c39e3569138414560c76defc0d4
Successfully built imagenette-experiments


In [3]:
from imagenette_experiments.train_utils import *

In [4]:
from kornia.contrib import MaxBlurPool2d

In [5]:
from fastai.basic_train import *
from fastai.vision import *
# from fastai.script import *
from model_constructor.net import Net, act_fn
from model_constructor.layers import SimpleSelfAttention, ConvLayer

# ResBlock

In [6]:
class NewLayer(nn.Sequential):
    """Basic conv layers block"""
    def __init__(self, ni, nf, ks=3, stride=1,
            act=True,  act_fn=nn.ReLU(inplace=True),
            bn_layer=True, bn_1st=True, zero_bn=False,
            padding=None, bias=False, groups=1, **kwargs):

        if padding==None: padding = ks//2
        if ks==3 and nf!=ni*4:  # to be used for the "stem" of ResNet
          layers = [('Conv3x3', 
              nn.Conv2d(ni, ni*4, 3, padding=1, bias=bias, groups=ni)),
                    ('act_fn', act_fn),
                    ('Conv1x1',
              nn.Conv2d(ni*4, nf, 1, bias=bias, groups=1))
              ]
        else:
          layers = [('Conv{}x{}'.format(ks,ks), 
              nn.Conv2d(ni, nf, ks, stride=stride, padding=padding, bias=bias, groups=groups))]

        act_bn = [('act_fn', act_fn)] if act else []
        if bn_layer:
            bn = nn.BatchNorm2d(nf)
            nn.init.constant_(bn.weight, 0. if zero_bn else 1.)
            act_bn += [('bn', bn)]
        if bn_1st: act_bn.reverse()
        layers += act_bn
        super().__init__(OrderedDict(layers))

In [7]:
class NewResBlock(Module):
    def __init__(self, expansion, ni, nh, stride=1,
                 conv_layer=ConvLayer, act_fn=act_fn, bn_1st=True,
                 pool=nn.AvgPool2d(2, ceil_mode=True), sa=False, sym=False, zero_bn=True):
        nf,ni = nh*expansion,ni*expansion
        conv_layer = NewLayer
        self.reduce = noop if stride==1 else pool
        layers  = [(f"conv_0", conv_layer(ni, nh, 3, act_fn=act_fn, bn_1st=bn_1st)),
                   (f"conv_1", conv_layer(nh, nf, 3, zero_bn=zero_bn, act=False, bn_1st=bn_1st))
        ] if expansion == 1 else [
                   (f"conv_0", conv_layer(ni, nh, 1, act_fn=act_fn, bn_1st=bn_1st)),
                   (f"conv_1", conv_layer(nh, nh*4, 3, groups=nh, act_fn=act_fn, bn_1st=bn_1st)),
                   (f"conv_2", conv_layer(nh*4, nf, 1, zero_bn=zero_bn, act=False, bn_1st=bn_1st))
        ]
        if sa: layers.append(('sa', SimpleSelfAttention(nf,ks=1,sym=sym)))
        self.convs = nn.Sequential(OrderedDict(layers))
        self.idconv = noop if ni==nf else conv_layer(ni, nf, 1, act=False, bn_1st=bn_1st)
        self.merge = act_fn

    def forward(self, x):
        o = self.reduce(x)
        return self.merge(self.convs(o) + self.idconv(o))

# Model Constructor

In [8]:
model = Net(c_out=10, layers=[3,6,8,3], expansion=4)
model.block = NewResBlock
model.conv_layer = NewLayer # for the stem
pool = MaxBlurPool2d(3, True)
model.pool = pool
model.stem_pool = pool
model.stem_sizes = [3,32,64,64]
model.act_fn = Mish()
model.sa = True
res = []

# Runs and results

## size=128


In [22]:
# not for stem
epochs = [5]*5
for e in epochs:
    mixup=0 if e<=20 else 0.2
    learn = get_learn(model=model, size=128, bs=32, mixup=mixup)
    learn.fit_fc(e, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res)) 

data path   /root/.fastai/data/imagewoof2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/u

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.926666,1.852383,0.400865,0.859506,01:31
1,1.635753,1.545834,0.546704,0.930771,01:31
2,1.448534,1.349256,0.637312,0.949351,01:30
3,1.313903,1.230567,0.693561,0.960041,01:30
4,1.087285,1.060132,0.768898,0.974803,01:30


data path   /root/.fastai/data/imagewoof2


...

data path   /root/.fastai/data/imagewoof2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/u

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.874006,1.773460,0.433698,0.882158,01:29
1,1.626993,1.531710,0.564266,0.925681,01:30
2,1.446367,1.299589,0.665055,0.953423,01:30
3,1.306742,1.216297,0.705778,0.959786,01:29
4,1.082566,1.053554,0.780606,0.973785,01:30


[0.768898, 0.779842, 0.781115, 0.781369, 0.780606] 0.7783660054206848


In [10]:
# depthwise also for stem [3,32,64,64]
epochs = [5]*5
for e in epochs:
    mixup=0 if e<=20 else 0.2
    learn = get_learn(model=model, size=128, bs=16, mixup=mixup)
    learn.fit_fc(e, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res)) 

data path   /root/.fastai/data/imagewoof2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/u

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.906552,1.751375,0.438534,0.884449,04:05
1,1.593676,1.524974,0.546449,0.928735,04:05
2,1.442727,1.363466,0.636803,0.949351,04:06
3,1.226924,1.165814,0.734029,0.971749,04:06
4,1.061580,1.017459,0.814202,0.977093,04:06


data path   /root/.fastai/data/imagewoof2


...

data path   /root/.fastai/data/imagewoof2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/u

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.886371,1.799726,0.415373,0.877577,04:05
1,1.655607,1.534360,0.534487,0.926953,04:04
2,1.485527,1.345568,0.639094,0.948842,04:04
3,1.305475,1.203766,0.715195,0.963095,04:04
4,1.093475,1.035459,0.796895,0.978112,04:04


[0.815984, 0.814202, 0.817765, 0.816493, 0.796895] 0.8122677564620971


In [25]:
res, '5 mean: {}'.format(sum(res)/5)

([0.7688979506492615,
  0.77984219789505,
  0.7811148166656494,
  0.7813693284988403,
  0.7806057333946228],
 '5 mean: 0.7783660054206848')

In [26]:
epochs = [20]*5
for e in epochs:
    mixup=0 if e<=20 else 0.2
    learn = get_learn(model=model, size=128, bs=32, mixup=mixup)
    learn.fit_fc(e, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/u

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.949271,1.823427,0.420972,0.866124,01:30
1,1.642933,1.505736,0.563248,0.926953,01:29
2,1.459789,1.360561,0.628913,0.946042,01:30
3,1.337876,1.230123,0.699415,0.954950,01:30
4,1.246346,1.208583,0.699669,0.960041,01:29
5,1.134926,1.115217,0.755409,0.968440,01:29
6,1.088735,1.057945,0.770171,0.974039,01:30
7,1.029235,1.032997,0.784169,0.969458,01:30
8,0.982295,1.000211,0.791550,0.979384,01:30
9,0.964776,0.984725,0.805548,0.973785,01:30


data path   /root/.fastai/data/imagewoof2


...

data path   /root/.fastai/data/imagewoof2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/u

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.891569,1.710517,0.453041,0.897684,01:30
1,1.643553,1.443666,0.594808,0.944261,01:30
2,1.431929,1.319665,0.651565,0.951387,01:30
3,1.315495,1.235722,0.687961,0.961313,01:30
4,1.192190,1.121433,0.747773,0.963858,01:30
5,1.138651,1.067773,0.769152,0.971494,01:30
6,1.077378,1.048082,0.780860,0.974548,01:29
7,1.036416,1.075319,0.769152,0.971749,01:30
8,0.976910,0.991011,0.804021,0.974803,01:30
9,0.939330,1.024645,0.792314,0.975312,01:30


[0.768898, 0.779842, 0.781115, 0.781369, 0.780606, 0.856197, 0.861033, 0.859252, 0.859252, 0.861288] 0.818885213136673


In [27]:
res[5:], sum(res[5:])/5

([0.8561974763870239,
  0.8610333204269409,
  0.8592517375946045,
  0.8592517375946045,
  0.8612878322601318],
 0.8594044208526611)

In [14]:
sum(res[5:])/5

0.8662255048751831

In [4]:
res = [0.861542, 0.859252, 0.859252, 0.863324, 0.864851]
sum(res)/5

0.8616442000000001

## size=192

In [9]:
epochs = [5]*5
for e in epochs:
    mixup=0 if e<=20 else 0.2
    learn = get_learn(model=model, size=192, bs=32, mixup=mixup)
    learn.fit_fc(e, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/u

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.852821,1.733783,0.456350,0.890303,02:22
1,1.582649,1.475512,0.574446,0.934843,02:22
2,1.386568,1.275406,0.674981,0.954950,02:23
3,1.230152,1.134809,0.741919,0.964113,02:23
4,1.015104,0.979367,0.820311,0.980148,02:23


/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value)


...

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/u

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.830721,1.724038,0.471876,0.904301,02:23
1,1.564874,1.366736,0.630949,0.947569,02:23
2,1.366136,1.275826,0.679053,0.954441,02:23
3,1.216263,1.147997,0.741410,0.968185,02:23
4,1.018347,0.993846,0.811148,0.978621,02:23


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "


[0.820311, 0.812166, 0.809112, 0.804785, 0.811148] 0.8115041971206665


In [15]:
epochs = [20]*5
for e in epochs:
    mixup=0 if e<=20 else 0.2
    learn = get_learn(model=model, size=192, bs=32, mixup=mixup)
    learn.fit_fc(e, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/u

RuntimeError: ignored